In [1]:
import pandas as pd
import os
from tqdm import tqdm
import json

titlePath = '../dataset/Data_TitleInfo2.csv'
studentPath = '../dataset/Data_StudentInfo.csv'
submitPath = '../dataset/Data_SubmitRecord'


# 获取题目数据
def getTitle():
    title = pd.read_csv(titlePath)
    return title

# 获取学生数据
def getStudent():
    student = pd.read_csv(studentPath)
    return student.drop(columns=['index'])

# 获取提交数据（班级号）
def getSubmit(classId=1):
    submit = pd.read_csv(os.path.join(submitPath, 'SubmitRecord-Class' + str(classId) + '.csv'))
    return submit.drop(columns=['index'])

# 所有提交数据
def getAllSubmit():
    submitRecords = []
    for i in range(1, 16):
        submitRecords.append(getSubmit(i))
    return pd.concat(submitRecords)

In [2]:
# 简单清洗并合并数据
def cleanMerge():
    title = getTitle()
    student = getStudent()
    submit = getAllSubmit()

    print(title.shape, student.shape, submit.shape)
    submit = submit.drop_duplicates()
    print(submit.shape)

    # 去除提交记录中的错误student_id
    # print(student['student_ID'].unique().shape)
    # print(submit['student_ID'].unique().shape)
    error_students = submit[~submit['student_ID'].isin(student['student_ID'])]
    # print('errorid',error_students.shape,error_students['student_ID'].iloc[0])
    submit = submit[submit['student_ID'] != error_students['student_ID'].iloc[0]]
    print(submit.shape)


    # 提交记录中，class错误的修正
    print('-----')
    print(submit['class'].unique())
    error_rows = submit[submit['class'] == 'class']
    for index,row in error_rows.iterrows():
        id = row['student_ID']
        # print(id)
        # print(id, sbms.shape, sbms['class'].unique()[0])
        submit.loc[index, 'class'] = submit[submit['student_ID'] == id]['class'].unique()[0]

    print(submit['class'].unique())

    # 检查题目id
    print('-----')
    print(title['title_ID'].unique().shape)
    print(submit['title_ID'].unique().shape)

    print('-----')
    # merge的时候发现，提交数据和题目数据中的score冲突，更改了题目数据中score->fullscore
    # 注意：题目数据使用的是TitleInfo2.csv
    print(submit.shape, submit.columns)
    merge1 = pd.merge(submit, student, on='student_ID')
    print(merge1.shape, merge1.columns)
    merge2 = pd.merge(merge1, title, on='title_ID')
    print(merge2.shape, merge2.columns)

    return merge2

In [3]:
df = cleanMerge()
df.to_csv('../dataset/Data_Merged.csv', index=False)


(38, 7) (1364, 4) (232818, 9)
(231812, 9)
(231811, 9)
-----
['Class1' 'class' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6' 'Class7'
 'Class8' 'Class9' 'Class10' 'Class11' 'Class12' 'Class13' 'Class14'
 'Class15']
['Class1' 'Class15' 'Class7' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6'
 'Class8' 'Class9' 'Class10' 'Class11' 'Class12' 'Class13' 'Class14']
-----
(38,)
(38,)
-----
(231811, 9) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID'],
      dtype='object')
(231811, 12) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID', 'sex', 'age', 'major'],
      dtype='object')
(231811, 18) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID', 'sex', 'age', 'major', 'fullscore',
       'knowledge_num', 'knowledge', 'sub_knowledge', 'knowledge2',
       'sub_knowledge2'],
      dtype='object')
